In [1]:
import cv2
from tensorflow.keras.models import load_model
import time

Fonction d'affichage des émoticônes :

In [10]:
def afficher_emote(statut):
    
    if statut == "Angry":
        img2 = angry
        
    elif statut == "Neutral":
        img2 = neutral
        
    elif statut == "Sad":
        img2 = sad
        
    elif statut == "Happy":
        img2 = happy
        
    elif statut == "Disgust":
        img2 = disgust
        
    elif statut == "Fear":
        img2 = fear
        
    elif statut == "Surprise":
        img2 = surprise
    
    else : img2 = rien
        
    # Création d'une région d'intérêt openCV au coin supérieur-gauche & de la taille du smiley :
    rows,cols,channels = img2.shape
    roi = im2[0:rows, 0:cols ]

    # Création d'un masque du smiley (et son masque inverse)
    img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 30, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)

    # Black-out du smiley dans la ROI :
    im_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
    
    # De l'image du smiley, on ne prend que le smiley en lui-même :
    img2_fg = cv2.bitwise_and(img2,img2,mask = mask)

    # Ajout du smiley dans le ROI et modification de l'image principale pour intégrer l'ajout :
    dst = cv2.add(im_bg,img2_fg)
    im2[0:rows, 0:cols ] = dst

    return

Programme de détection d'émotions :

In [13]:
model = load_model("./modele_emotions/")
sad = cv2.imread('./sad.png')
happy = cv2.imread('./happy.png')
rien = cv2.imread('./rien.png')
angry = cv2.imread('./angry.png')
disgust = cv2.imread('./disgust.png')
surprise = cv2.imread('./surprise.png')
neutral = cv2.imread('./neutral.png')
fear = cv2.imread('./fear.png')

# Facteur pour le redimensionnement de l'image
imgsize = 4
# Allumage de la webcam : 
camera = cv2.VideoCapture(0)
# Identification d'un visage :
classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Touche pour fermer, initialisée à 0, si elle passe à 27 avec le waitKey(), c'est que l'utilisateur a fait ESC
key=0
while key!=27:
    # Deux variables vont être continuellement remplies via la cam avec la méthode read().
    # rval est un booléen qui atteste du fait que les frames de la cam sont bien lues, im sera notre image vidéo.
    (rval, im) = camera.read()
    # On passe l'image en miroir
    im=cv2.flip(im,1,1)
    
    # Création d'une copie de l'image issue de la webcam
    # La prédiction se fera via im sans le smiley pour éviter qu'il soit reconnu comme un visage:
    im2 = im.copy()
    
    # Redimensionnement à la baisse de l'image récupérée de la webcam (640*480 => 160*120)
    # Objectif : soulaver le classifieur de haar.
    image_redim = cv2.resize(im, (im.shape[1] // imgsize,
                             im.shape[0] // imgsize))
        
    # On crée nos frames de visage reconnu par classifieur haar appliqué aux images issues des frames redimensionnées :
    face_rec = classifier.detectMultiScale(image_redim) 
    # print("face_rec=",face_rec)
    
    # Si le type de face_rec n'est pas tuple, cela veut dire que face_rec != () (càd qu'il n'est pas vide)
    if type(face_rec) != tuple: # faut-il aussi faire "and 0 not in face_rec[0]" ?
        # Pour chaque visage reconnu :
        for i in face_rec:
            # Le classifieur haar travaillait sur images réduites de facteur 4 mais pour découper
            # un médaillon rectangulaire qui correspond au visage détecté sur notre image webcam,
            # il faut à nouveau multiplier les données d'output du classifieur par le imsize :
            (x, y, l, w) = [v * imgsize for v in i]
            # On découpe dans l'image de la caméra avec les coordonnées obtenues grâce à haar.
            face_img = im[y:(y+w), x:(x+l)]
            # On redimensionne :
            face_redim=cv2.resize(face_img,(100,100))
            # On passe en gris :
            face_gray = cv2.cvtColor(face_redim, cv2.COLOR_RGB2GRAY)
            
            # En l'absence d'un modèle entraîné sur un dataset dûment data augmenté
            # avec des doublons d'images aux contrastes et luminosités variées, le
            # réglage de contraste et de brightness pour la prédiction est crucial.
            # Selon l'éclairage dans lequel tourne l'appli et la webcam, il faut régler
            # les deux paramètres suivants.
            # S'il fait sombre, brightness à 2 marche bien, sinon 1.
            
            contrast = 2
            brightness = 1
            face_contrast = cv2.addWeighted(face_gray, contrast, face_gray, 0, brightness)

            # On standardise la valeur des pixels de l'array :
            face_standardise= face_contrast.astype('float')/255

            # On reshape pour fonctionner avec le CNN :
            #face_reshaped = face_standardise.reshape(-1,48,48,1)
            
            face_resized = cv2.resize(face_standardise, dsize=(48,48))
            face_reshaped = face_resized.reshape(-1,48,48,1)
            
            afficher_emote(statut="rien")
            prediction = model.predict(face_reshaped)
            #cv2.putText(im2, str(prediction[0].round(2)), (0, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))

            # La prédiction sur image issue de flux vidéo fonctionne mais il faut
            # définir un seuil car on atteint pas des taux de certitude de 99%.
            if prediction[0].argmax() == 0 :
                #cv2.putText(im2, "Angry", (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
                afficher_emote(statut="Angry")
                cv2.putText(im2, "Reste calme et pose ce flingue !", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))

            
            elif prediction[0].argmax() == 1 :
                #cv2.putText(im2, "Disgust", (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
                afficher_emote(statut="Disgust")
                cv2.putText(im2, "Incroyable, tu as débloqué celle-ci ?!", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))


            elif prediction[0].argmax() == 2 :
                #cv2.putText(im2, "Fear", (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
                afficher_emote(statut="Fear")
                cv2.putText(im2, "Derrière-toi ! La COVID !", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))


            elif prediction[0].argmax() == 3 :
                #cv2.putText(im2, "Happy", (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
                afficher_emote(statut="Happy")
                cv2.putText(im2, "Il est content !", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))


            elif prediction[0].argmax() == 4 :
                #cv2.putText(im2, "Neutral", (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
                afficher_emote(statut="Neutral")
                cv2.putText(im2, "Neutre, RAS !", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))

                
            elif prediction[0].argmax() == 5 :
                #cv2.putText(im2, "Sad", (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
                afficher_emote(statut="Sad")
                cv2.putText(im2, "Ne sois pas triste, la vie est belle !", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))


            elif prediction[0].argmax() == 6 :
                #cv2.putText(im2, "Surprise", (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
                afficher_emote(statut="Surprise")
                cv2.putText(im2, "Surprise !", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))



    # On affiche im2 et pas im pour avoir l'image modifiée avec le smiley :
    cv2.imshow('Detecteur de masques',im2)
    key = cv2.waitKey(10)

camera.release()
cv2.destroyAllWindows()